<a href="https://colab.research.google.com/github/adammuhtar/semantic-information-retrieval/blob/main/notebooks/semantic_search_retrieve_rerank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Domain-Specific Semantic Search: Retrieve & Re-rank using Sentence Transformers**

Semantic search is a powerful technique in natural language processing (NLP): it enables users to find more relevant information by understanding the context and meaning behind their queries. Utilising encoder architectures from machine learning NLP techniques, semantic information retrieval allows the machine to extract the meaning and context of a user's query and match it with relevant documents, articles, or other data sources - this means that system can recognise synonyms, acronyms, and related terms, and can identify the relationship between different concepts and entities, providing a more nuanced and comprehensive understanding of the query. This technique is particularly useful when searching unstructured or semi-structured data, such as scientific papers or legal documents, where traditional NLP techniques such as keyword matching and rule-based approaches may not be sufficient to retrieve relevant information or could even return completely irrelevant results due to the variability, ambiguity, and complexity of language (the last one is a particularly hard problem in the case domain-specific corpus).

**[Retrieve & Re-Rank](https://www.sbert.net/examples/applications/retrieve_rerank/README.html)**

Retrieve & Re-Rank is one such method designed to tackle complex semantic search tasks, such as question-answer retrieval, by providing an efficient and accurate search process that retrieves the most relevant results for a given query.

The setup of this pipeline involves encoding the entire corpus in question onto a high-dimensional vector space. Once the corpus is embedded onto this vector space, the semantic search pipeline itself involves two-stages. The first starts with embedding the user's query onto this high-dimensional vector space, producing a question embedding that we then can compare against the rest of the corpus embeddings. The top N embeddings from the corpus that are most similar to the question embedding - using the cosine or dot product similarity method - are retrieved.

The first-stage bi-encoder retriever is designed to be efficient, especially when dealing with a potentially large corpus with millions of entries. This retriever on its own however, might still return irrelevant candidates if relied solely on cosine/dot product similarity metrics. To address this issue, a second stage in the pipeline involves a re-ranker based on a cross-encoder that scores the relevancy of all of these initial candidates for the given search query. The cross-encoder takes the query and a possible document and passes them simultaneously to a transformer network, scoring their alignment to indicate how relevant the document is for the given query.

The output of the pipeline is a ranked list of hits that are presented to the user. This list is generated by the re-ranker, which improves the final results for the user by substantially increasing the accuracy of the search process. While the retriever has to be efficient for large document collections with millions of entries, the Cross-Encoder can perform attention across the query and the document, resulting in higher performance.

**[SentenceTransformers](https://sbert.net)**

The Python framework to run a retrieve and re-rank pipeline is SentenceTransformers. This framework is built based on the [Sentence-BERT: Sentence Embeddings using Siamese BERT-Networks](https://arxiv.org/abs/1908.10084) paper that extends the BERT architecture - Bidirectional Encoder Representations from Transformers, a family of masked-language models introduced in 2018 by researchers at Google to learn and generate fixed-length vector representations of variable-length sentences - to derive semantically meaningful sentence embeddings that can be compared using cosine-similarity. Sentence-BERT (SBERT) is a modification of the standard pre-trained BERT network that uses siamese and triplet networks to create sentence embeddings.

This notebook explores the use of SentenceTransformers to build a domain-specific semantic information retrieval system, based on user provided input/queries.

---
*References:*

* Reimers, N. & Gurevych, I. (2019). Sentence-BERT: Sentence Embeddings using Siamese BERT-Networks. *arXiv* preprint arXiv:1908.10084.
---

## **Table of Contents**

* [1. Notebook setup](#section-1)
* [2. Load corpus](#section-2)
* [3. Load encoders and generate corpus embeddings](#section-3)
* [4. Lexical vs semantic search](#section-4)

## 1. Notebook Setup <a name="section-1"></a>

This notebook is run using Google Colaboratory (Colab) - Colab is Google's implementation of [Jupyter Notebooks](https://jupyter.org/). This notebook will require the following package(s) to be installed:
* `python==3.9.16`
* `datasets==3.7.1`
* `rank_bm25==1.22.4`
* `sentence-transformers==1.4.4`
* `torch==2.0.0+cu118`
* `transformers==4.28.1`

The `datasets`, `rank_bm25`, `sentencepiece`, `sentence-transformers`, and `transformers` libraries will need to be manually installed into the Colab environment (pip install by running a shell command). This Colab notebook could be run without any hardware accelerators, although running this with the Tesla T4 GPU (16 GB GDDR6 @ 320 GB/s) provided for free by Google may be helpful to access higher RAM runtimes.

In [ ]:
# Query GPU device status/details
!nvidia-smi

Tue Apr 18 21:00:21 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P8    14W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Check IP address details if there are restrictions running non-local servers
!curl ipinfo.io

{
  "ip": "35.204.59.50",
  "hostname": "50.59.204.35.bc.googleusercontent.com",
  "city": "Groningen",
  "region": "Groningen",
  "country": "NL",
  "loc": "53.2192,6.5667",
  "org": "AS396982 Google LLC",
  "postal": "9711",
  "timezone": "Europe/Amsterdam",
  "readme": "https://ipinfo.io/missingauth"
}

In [ ]:
!pip install datasets rank_bm25 sentence-transformers --quiet --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 66.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 27.3 MB/s eta 0:

In [ ]:
# Standard library imports
import string

# Third-party imports
from datasets import load_dataset
import numpy as np
import pandas as pd
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer, CrossEncoder, util
from sklearn.feature_extraction import _stop_words
import torch
from tqdm.autonotebook import tqdm

In [ ]:
# Check available GPUs for computation
if torch.cuda.is_available():
    num_gpus = torch.cuda.device_count()
    # Print details of all available GPUs
    for i in range(num_gpus):
        gpu_props = torch.cuda.get_device_properties(i)
        print(f"Device details for GPU {i+1}:")
        print(f"* Name: {gpu_props.name}")
        print(f"* Memory size: {round(gpu_props.total_memory / 1024**3, 2)} GB")
        if i == num_gpus-1:
            continue
        else:
            print("-"*79)
    # Get the currently active GPU device and print its name and memory size
    active_gpu = torch.cuda.current_device()
    active_gpu_props = torch.cuda.get_device_properties(active_gpu)
    print("="*79)
    print(f"Currently active GPU device: {active_gpu_props.name}")
    print(f"Memory size: {round(active_gpu_props.total_memory / 1024**3, 2)} GB")
    print("="*79)
else:
    print("No GPU devices found.")

Device details for GPU 1:
* Name: Tesla T4
* Memory size: 14.75 GB
Currently active GPU device: Tesla T4
Memory size: 14.75 GB


## 2. Load Corpus <a name="section-2"></a>

The corpus used for this notebook is sourced from the [`financial-news-articles`](https://huggingface.co/datasets/ashraq/financial-news), which itself was sourced from a Kaggle dataset [US Financial News Articles](https://www.kaggle.com/datasets/jeet2016/us-financial-news-articles). The dataset contains financial news articles from Bloomberg, CNBC, Reuters, Wall Street Journal, and Fortune, starting from January 2018 up to May 2018.

In [ ]:
from datasets import load_dataset

fin_news_raw = load_dataset("ashraq/financial-news-articles")
print("-"*79, "\nfinancial-news-articles data structure:")
fin_news_raw

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/306242 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/ashraq___parquet/ashraq--financial-news-articles-2c6ba3fd2690414f/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

------------------------------------------------------------------------------- 
financial-news-articles data structure:


DatasetDict({
    train: Dataset({
        features: ['title', 'text', 'url'],
        num_rows: 306242
    })
})

In [ ]:
# Example entry in dataset
fin_news_raw["train"][1000]

{'title': 'BRIEF-Positive Profit Warning For 2018 From As Pro Kapital Grupp',
 'text': 'Jan 17 (Reuters) - PRO KAPITAL GRUPP AS :\n* POSITIVE PROFIT WARNING FOR 2018 FROM AS PRO KAPITAL GRUPP\n* SAYS ESTIMATES TURNOVER AND NET PROFIT OF COMPANY TO INCREASE SIGNIFICANTLY IN 2018\n* SAYS PROJECTED REVENUES IN RANGE 35-40 MILLION EUR , WHICH IS AN INCREASE OF 200-250% COMPARED TO 2017\n* SAYS CONCLUDE 2018 WITH A POSITIVE NET RESULT IN RANGE OF 2-3 MILLION EUR\n* SAYS CHANGE IN ITS DIVIDEND POLICY AND HAS JUST PAID OUT DIVIDENDS OF 0,015EUR PER SHARE Source text : bit.ly/2DqGSqU Further company coverage: (Gdynia Newsroom)\n ',
 'url': 'https://www.reuters.com/article/brief-positive-profit-warning-for-2018-f/brief-positive-profit-warning-for-2018-from-as-pro-kapital-grupp-idUSFWN1PC0YU'}

In [ ]:
# Create list containing entire corpus
fin_news = []
for i in range(len(fin_news_raw["train"])):
    fin_news.append(fin_news_raw["train"][i]["text"])
print(f"Size of corpus: {len(fin_news)}")

Size of corpus: 306242


## 3. Load encoders and generate corpus embeddings <a name="section-3"></a>

The core algorithm of this approach is a cooperative retrieve-and-rerank approach uses twin networks for initial retrieval and a cross-encoder component for smarter ranking. The framework is based on a cooperative retrieve-and-rerank approach that combines:
1. Twin networks (i.e. a bi-encoder) to separately encode all items of a corpus, enabling efficient initial retrieval, and
2. Cross-encoder component for a more nuanced (i.e., smarter) ranking of the retrieved small set of items.


In [ ]:
# Choose which model to run
biencoder_model = "multi-qa-mpnet-base-dot-v1" #@param ["multi-qa-mpnet-base-dot-v1", "all-mpnet-base-v2", "multi-qa-distilbert-cos-v1", "multi-qa-MiniLM-L6-cos-v1", "all-distilroberta-v1", "all-MiniLM-L12-v2", "all-MiniLM-L6-v2"]
crossencoder_model = "cross-encoder/ms-marco-MiniLM-L-6-v2" #@param ["cross-encoder/ms-marco-MiniLM-L-12-v2", "cross-encoder/ms-marco-MiniLM-L-6-v2", "cross-encoder/ms-marco-MiniLM-L-4-v2", "cross-encoder/ms-marco-MiniLM-L-2-v2"]

# Initialise bi-encoder model
bi_encoder = SentenceTransformer("multi-qa-mpnet-base-dot-v1")
bi_encoder.max_seq_length = 512 #@param [256, 512]
top_k = 32

#The bi-encoder will retrieve 100 documents. We use a cross-encoder, to re-rank the results list to improve the quality
cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

In [ ]:
# Encode entire corpus into vector space using bi-encoder.
corpus_embeddings = bi_encoder.encode(
    fin_news,
    convert_to_tensor=True,
    show_progress_bar=True
)

Batches:   0%|          | 0/9571 [00:00<?, ?it/s]

In [ ]:
corpus_embeddings

tensor([[ 0.1497, -0.4827, -0.1416,  ..., -0.1339, -0.3964,  0.1694],
        [-0.4322, -0.1381, -0.2599,  ..., -0.3796, -0.0301, -0.1303],
        [-0.3534,  0.2285, -0.2826,  ..., -0.1742, -0.2147, -0.2111],
        ...,
        [ 0.2540, -0.0929, -0.2965,  ..., -0.1530, -0.0027, -0.1808],
        [-0.1619, -0.0090, -0.2351,  ...,  0.1010, -0.2537, -0.2906],
        [ 0.1999, -0.5149, -0.0626,  ..., -0.1142, -0.0490,  0.0746]],
       device='cuda:0')

We also set up an Okapi BM25 tokeniser pipeline to tokenise the corpus, enabling us to compare lexical search performance with semantic search via retrieve and re-rank.

In [ ]:
# We lower case our text and remove stop-words from indexing
def bm25_tokeniser(text: str) -> list:
    """
    Tokenises text for Okapi BM25 indexing by lowercasing the text, removing
    punctuation, and removing stop-words.

    Args:
        * text (`str`): Input string to be tokenised
    
    Returns:
        * `list`: A list of tokens that have been lowercased, stripped of
        punctuation, and filtered of stop-words
    """
    tokenised_doc = []
    for token in text.lower().split():
        token = token.strip(string.punctuation)
        if len(token) > 0 and token not in _stop_words.ENGLISH_STOP_WORDS:
            tokenised_doc.append(token)
    return tokenised_doc

# Tokenise all news articles in the corpus
tokenised_corpus = []
for news in tqdm(fin_news):
    tokenised_corpus.append(bm25_tokeniser(news))

bm25 = BM25Okapi(tokenised_corpus)

  0%|          | 0/306242 [00:00<?, ?it/s]

## 4. Lexical vs semantic search

This section compares search performance of three methods:
1. Okapi BM25 lexical search, which ranks documents based on query term frequency regardless of proximity;
2. Semantic search using a bi-encoder retrieval model that encodes queries into vector space and retrieves document embeddings via cosine similarity; and
3. Semantic search with bi-encoder retrieval and cross-encoder re-ranker model, which further refines the previous step by passing query and article pairs through a transformer for relevance classification.

In [ ]:
# Function to search news articles that answers the query
def search(n: int = 3, top_k: int = 32, search_scores: bool = False):
    """
    Searches all financial news articles for passages that answer the query via:
    * Okapi BM25 lexical search, a bag-of-words retrieval function that ranks a
    set of documents based on the query terms appearing in each document,
    regardless of their proximity within the document.
    * Semantic search via bi-encoder retrieval model, which encodes the query
    into vector space and retrieves the document embeddings that are close in
    vector space through cosine similarity search.
    * Semantic search via bi-encoder retrieval and cross-encoder re-ranker model,
    which adds to the previous step by simultaneously passing through the query
    and potential article pairs through a transformer for an entailment
    classifier on how relevant the passage is for the given query.

    Args:
        * n (`int`): Number of search results to return
        * top_k (`int`): Retrieve top-k matching entries
        * search_scores (`bool`): Set to True if user wants ranked results
        dictionary
    """
    query = input("Query: ")
    print("="*100)

    # Lexical search with Okapi BM25
    bm25_scores = bm25.get_scores(bm25_tokeniser(query))
    top_n = np.argpartition(bm25_scores, -5)[-5:]
    bm25_hits = [{"corpus_id": idx, "score": bm25_scores[idx]} for idx in top_n]
    bm25_hits = sorted(bm25_hits, key=lambda x: x["score"], reverse=True)
    print(f"Top-{n} Lexical Search (Okapi BM25) Hits:")
    for hit in bm25_hits[0:n]:
        print(
            f"* Search score: {round(hit['score'], 3)} |",
            fin_news[hit["corpus_id"]].replace("\n", " ")
        )
    print("-"*100)

    # Semantic search via bi-encoder retrieval through cosine similarity
    question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    question_embedding = question_embedding.cuda()
    hits = util.semantic_search(
        query_embeddings=question_embedding,
        corpus_embeddings=corpus_embeddings,
        top_k=top_k
    )
    hits = hits[0]  # Get the hits for the first query

    # Cross-encoder re-ranking of retrieved documents
    cross_input = [[query, fin_news[hit["corpus_id"]]] for hit in hits]
    cross_scores = cross_encoder.predict(cross_input)

    # Sort results by the cross-encoder scores
    for idx in range(len(cross_scores)):
        hits[idx]["cross-score"] = cross_scores[idx]

    # Print top-5 hits from bi-encoder retrieval
    print(f"Top-{n} Semantic Search (Bi-Encoder Retrieval) Hits:")
    hits = sorted(hits, key=lambda x: x["score"], reverse=True)
    for hit in hits[0:n]:
        print(
            f"* Search score: {round(hit['score'], 3)} |",
            fin_news[hit["corpus_id"]].replace("\n", " ")
        )
    print("-"*100)

    # Print top-5 hits from cross-encoder re-ranker
    print(f"Top-{n} Semantic Search (Bi-Encoder + Cross-Encoder) Hits:")
    hits = sorted(hits, key=lambda x: x["cross-score"], reverse=True)
    for hit in hits[0:n]:
        print(
            f"* Search score: {round(float(hit['cross-score']), 3)} |",
            fin_news[hit["corpus_id"]].replace("\n", " ")
        )
    print("-"*100)
    if search_scores:
        return hits[:50], bm25_hits[:50], question_embedding
    else:
        return None

### Test 1: JPMorgan share price movement after earnings call

In [ ]:
search()

Query: JPMorgan share price movement after earnings call
Top-3 Lexical Search (Okapi BM25) Hits:
* Search score: 18.584 | January 4, 2018 / 9:43 PM / Updated 8 minutes ago BRIEF-JPMorgan Chase Says Warrant Exercise Price Will Be Reduced To $41.764/Share From $41.834/Share Reuters Staff  Jan 4 (Reuters) - JPMorgan Chase & Co:  * JPMORGAN CHASE ANNOUNCES ADJUSTMENTS TO WARRANT EXERCISE PRICE AND WARRANT SHARE NUMBER  * JPMORGAN CHASE & CO - WARRANT EXERCISE PRICE WILL BE REDUCED TO $41.764/SHARE FROM $41.834/SHARE  * JPMORGAN CHASE & CO - WARRANT SHARE NUMBER WILL BE INCREASED TO 1.02 FROM 1.01 Source text for Eikon: Further company coverage:
* Search score: 17.123 | NEW YORK--(BUSINESS WIRE)-- JPMorgan Chase Financial Company LLC (the “ Issuer ”) today announces the pricing of its public offering of $350 million of cash-settled equity linked notes linked to the common stock of Voya Financial, Inc. (“ Voya ”) due May 1, 2023 (the “ Notes ”). JPMorgan Chase & Co. will fully and unconditio

### Test 2: Credit Suisse losses in February

*Comments*: The top hit for lexical search was a news article from March, whereas, retrieve and re-rank returns an article from February.

In [ ]:
search()

Query: Credit Suisse losses in February
Top-3 Lexical Search (Okapi BM25) Hits:
* Search score: 25.137 | March 14, 2018 / 10:04 PM / Updated 7 minutes ago BRIEF-Credit Suisse is sued in U.S. over investor losses from inverse VIX ETNs Reuters Staff 1 Min Read March 14 (Reuters) - Credit suisse is sued over losses related to credit suisse velocityshares daily inverse vix short term exchange traded notes — u.s. Court filing Lawsuit against credit suisse, ceo tidjane thiam and cfo david mathers is filed in u.s. District court in manhattan — court filing Lawsuit seeks class action status for investors who bought the etns between january 29 and february 5, 2018 Lawsuit accuses defendants of making false or misleading statements about the etns, causing investors to buy them at inflated prices Lawsuit accuses credit suisse of manipulating the etns by liquidating its holdings in various financial products to avoid a loss Lawsuit says the etn’s price fell by nearly 90 percent in less than three 

### Test 3: Changes in macroprudential regulation from UK central bank

*Comments*: The top hit from lexical search is an article about macroprudential regulation changes from Indonesia's central bank, whereas retrieve and rerank accurately identifies the Bank of England as the UK central bank and returns article about the BoE's changes in macroprudential regulation.

In [ ]:
search()

Query: Changes in macroprudential regulation from UK central bank
Top-3 Lexical Search (Okapi BM25) Hits:
* Search score: 26.102 | JAKARTA (Reuters) - Indonesia’s central bank will give banks greater flexibility in managing liquidity and credit in new rules announced on Thursday that are aimed at getting banks to lend more, officials said. FILE PHOTO: People walk in the courtyard of Indonesia's central bank, Bank Indonesia, in Jakarta, Indonesia September 22, 2016.REUTERS/Iqro Rinaldi/File Photo Bank Indonesia (BI) cut its benchmark policy rate 200 basis points in 2016 and 2017, but banks’ loan growth has remained well below the double-digit rates of earlier years. Annual bank credit grew 8.2 percent in February. The banking industry tends to follow an economic cycle and the new instruments will act as tools to help guide them to counter the cycle, head of BI’s macroprudential department Filianingsih Hendarta told reporters. In the current “lethargic condition”, Hendarta said credit gr

### Test 4: UK central bank interest rates

*Comments*: Retrieve and rerank returns an article about Bank of England - correctly identifying it as the UK central bank - whereas lexical search's top hit has a mix of commentaries about monetary policy from the ECB and BoE.

In [ ]:
search()

Query: UK central bank interest rates
Top-3 Lexical Search (Okapi BM25) Hits:
* Search score: 21.149 | April 25, 2018 / 12:00 PM / a day ago Commentary: Once it gets going, ECB may be more reliable rate hiker than BoE Jamie McGeever 5 As the global shift towards higher interest rates moves up a gear, attention is turning to Europe’s two major central banks. Central Bank Governors Mario Draghi of the European Central Bank (ECB) and Mark Carney of the Bank of England attend ECB's Central Bank Communications Conference in Frankfurt, Germany, November 14, 2017. REUTERS/Kai Pfaffenbach Current market pricing strongly suggests the Bank of England will tighten further and faster than European Central Bank. But the reality may turn out to be quite different. Having hiked rates once last year to counter a slump in the pound and the resulting rise in inflation, the BoE is further down the road of policy “normalization” than the ECB, albeit only slightly. Money markets show investors expect the E